# Model
The goal of this document is to set up a pipeline containing data preprocessing (feature selection and onwards), model training, and model testing.

## Importing dependencies

In [1]:
DEBUG = False

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
import h5py

import json

from tensorflow.keras import backend as keras
from keras.utils import to_categorical

import matplotlib.pyplot as plt

import numpy as np

import os

import pandas as pd

import pickle

import random

from sklearn import ensemble, preprocessing, svm
from sklearn.dummy import DummyClassifier
from sklearn.metrics import roc_auc_score, average_precision_score, recall_score 
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, ShuffleSplit
from sklearn.utils import shuffle, class_weight

import sqlite3

import tempfile

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, AveragePooling1D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import DepthwiseConv2D, AveragePooling2D, SeparableConv2D
from tensorflow.keras.layers import LSTM, GRU, RNN
from tensorflow.keras.losses import MAE
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1, l2

# from tensorflow.compat.v1.keras.backend import set_session as keras_set_session
from tensorflow.python.client import device_lib
# visible_devices = '1' #this is the GPU number, this is GPU0 ‘0’ or GPU1 ‘1’
# memory_fraction = 1.0 #This will allow 20% of the GPU memory to be allocated to your process, pick this number large enough for your script but also not too large so others can still do things.
# os.environ["CUDA_VISIBLE_DEVICES"] = visible_devices
# gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=memory_fraction, allow_growth=True) 
# tf_session = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))
# keras_set_session(tf_session)

import random
import csv

Using TensorFlow backend.


In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 7896707831425662554,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 10480091524725652059
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 6784260087677248722
 physical_device_desc: "device: XLA_GPU device",
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14613293312
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13275720127576791489
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [4]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

## Read in data(base)
Load in SQLite database and fetch all tokens (recordings) and the extracted features.

In [5]:
#
#   CONNECT TO LOCAL DATABASE
#
def create_db_connection(db_file_name):
    conn = None

    try:
        conn = sqlite3.connect(db_file_name)
    except sqlite3.Error as e:
        print(e)

    return conn

In [6]:
df_TUEP, df_TUAB, df_TUSZ = None, None, None

In [7]:
#TUEP
conn = create_db_connection('/mnt/disks/data/files/TUEP_files/eeg_recordings_TUEP.db')


query = """SELECT patients.patient_id, sessions.session_id, tokens.token_id, patients.diagnosis, sessions.electrode_setup, tokens.recording_duration, tokens.sampling_freq, tokens.len_of_samples, tokens.file_name, tokens.file_path
           FROM patients 
           
           INNER JOIN sessions ON patients.patient_id == sessions.patient_id 
           INNER JOIN tokens ON sessions.patient_id == tokens.patient_id AND sessions.session_id == tokens.session_id"""
cur = conn.cursor()
df_TUEP = pd.read_sql(query, conn)

#TUAB
conn = create_db_connection('/mnt/disks/data/files/TUAB_files/eeg_recordings_TUAB.db')


query = """SELECT patients.patient_id, sessions.session_id, tokens.token_id, patients.diagnosis, sessions.electrode_setup, tokens.recording_duration, tokens.sampling_freq, tokens.len_of_samples, tokens.file_name, tokens.file_path
           ,patients.patient_train_or_test FROM patients 
           INNER JOIN sessions ON patients.patient_id == sessions.patient_id 
           INNER JOIN tokens ON sessions.patient_id == tokens.patient_id AND sessions.session_id == tokens.session_id"""
cur = conn.cursor()
df_TUAB = pd.read_sql(query, conn)

#TUSZ
conn = create_db_connection('/mnt/disks/data/files/TUSZ_files/eeg_recordings_TUSZ.db')

query = """SELECT patients.patient_id, sessions.session_id, tokens.token_id, tokens.number_of_windows, tokens.diagnosis, sessions.electrode_setup, tokens.recording_duration, tokens.sampling_freq, tokens.len_of_samples, tokens.file_name, tokens.file_path
            FROM patients 
           INNER JOIN sessions ON patients.patient_id == sessions.patient_id 
           INNER JOIN tokens ON sessions.patient_id == tokens.patient_id AND sessions.session_id == tokens.session_id"""
cur = conn.cursor()
df_TUSZ = pd.read_sql(query, conn)

#TUSL
conn = create_db_connection('/mnt/disks/data/files/TUSL_files/eeg_recordings_TUSL.db')

query = """SELECT patients.patient_id, sessions.session_id, tokens.token_id, tokens.number_of_windows, tokens.diagnosis, sessions.electrode_setup, tokens.recording_duration, tokens.sampling_freq, tokens.len_of_samples, tokens.file_name, tokens.file_path
            FROM patients 
           INNER JOIN sessions ON patients.patient_id == sessions.patient_id 
           INNER JOIN tokens ON sessions.patient_id == tokens.patient_id AND sessions.session_id == tokens.session_id"""
cur = conn.cursor()
df_TUSL = pd.read_sql(query, conn)

In [8]:
df_dict = {"TUEP": df_TUEP, "TUAB": df_TUAB, "TUSZ": df_TUSZ, "TUSL": df_TUSL}
h5f_dict = {"TUEP": "/mnt/disks/data/files/TUEP_files/raw_data_TUEP.h5", "TUAB": "/mnt/disks/data/files/TUAB_files/raw_data_TUAB.h5", "TUSZ": "/mnt/disks/data/files/TUSZ_files/raw_data_TUSZ.h5", "TUSL": "/mnt/disks/data/files/TUSL_files/raw_data_TUSL.h5"}
plus_edf_dict = {"TUEP": False, "TUAB": True, "TUSZ": True, "TUSL": True}
class1_dict = {"TUEP": "epilepsy", "TUAB": "abnormal", "TUSZ": "seiz", "TUSL": "slow"}
class2_dict = {"TUEP":"no_epilepsy", "TUAB": "normal","TUSZ": "bckg", "TUSL": "bckg"}
multiple_labels_dict = {"TUEP": False, "TUAB": False, "TUSZ": True, "TUSL": True}

model_dict = {"TUEP": "tuep_baseline_a.hdf5", "TUAB": "tuab_baseline_a_2.hdf5", "TUSZ": "tusz_baseline_a_3.hdf5"}

In [9]:
#Avoid data leakage
dataset1 = "TUEP"
dataset2 = "TUAB"

patient_ids1 = np.unique(df_dict[dataset1]["patient_id"])
patient_ids2 = np.unique(df_dict[dataset2]["patient_id"])
intersection = list(set(patient_ids1) & set(patient_ids2))
print(len(intersection))
print(len(patient_ids1))
print(len(patient_ids2))
#df_TUAB = df_TUAB[df_TUAB["patient_id"].isin(intersection) == False]
#print(len(np.unique(df_TUAB["patient_id"])))
if len(patient_ids1) > len(patient_ids2):
    dataset = dataset1
else:
    dataset = dataset2

df_dict[dataset] = df_dict[dataset][df_dict[dataset]["patient_id"].isin(intersection) == False]
print(len(np.unique(df_dict[dataset]["patient_id"])))

121
200
1240
1119


### Overarching function

In [10]:
def window_sum(l):
    window_sum = 0
    for windows in l:
        windows = windows.split(" ")
        for window in windows:
            if not window == '':
                window_sum += int(window)
    return window_sum

In [11]:
def window_list(l):
    window_list = []
    for windows in l:
        window_element = []
        windows = windows.split(" ")
        for window in windows:
            if not window == '':
                window_element.append(int(window))
        window_list.append(window_element)
    return window_list

In [12]:
def total_windows(l):
    total = 0
    for window_element in l:
        for window in window_element:
            total += window
    return total

In [13]:
#
#   CALCULATE THE CUMULATIVE NUMBER OF TOKENS/SECONDS OF RAW DATA PER PATIENT
#
def get_tokens_cumsum(df, name, patient_ids):

    #       Iterate over patient_ids and keep track of number of tokens per patient
    patient_tokens = []
    for curr_patient_id in patient_ids:
        curr_patient_tokens = df[df["patient_id"] == curr_patient_id]
        if multiple_labels_dict[name]:
            curr_patient_windows = window_sum(df[df["patient_id"] == curr_patient_id]['number_of_windows'])
            patient_tokens.append(int(curr_patient_windows))
        else:
            curr_patient_windows = np.floor(df[df["patient_id"] == curr_patient_id]["recording_duration"] / 10)
            patient_tokens.append(curr_patient_windows.sum())        
        
    patient_tokens = np.asarray(patient_tokens)

    #       Calculate cumulative number of recordings 
    patient_tokens_cumsum = np.cumsum(patient_tokens)
    
    return patient_tokens_cumsum

In [14]:
#
#   EXTRACT VALUES AND LABELS FROM TRAIN/VALIDATION/TEST DATASET; TAKES FEATURES TO BE SELECTED AS ARGUMENTS TO FORWARD TO FEATURE SELECTION ALGORITHM
#
def data_to_values_and_labels(dfs, name):
    dfs_values, dfs_labels = [], []
        
    
    # Read in diagnosis and features/feature images from database
    for df in dfs:
        values = [] # (NR_OF_SAMPLE)
        labels = [] # (NR_OF_SAMPLES,)
            
        rows = df.iterrows()
        for row_index, row in rows:
            # Return list of filenames to read in with generator
            #       Extract file path and name from database
            file_name = row["file_name"]
            file_path = row["file_path"]

            #       Add file path and name to values array
            values.append(f"{'data'}{file_path}{file_name}")

            #       Add patient diagnosis as label to labels array
            if multiple_labels_dict[name]:#TODO: make sure this works
                label_list = []
                diagnoses = row["diagnosis"]
                diagnoses = diagnoses.split(" ")
                for diagnosis in diagnoses:
                    if diagnosis == class1_dict[name]:
                        label_list.append(1)
                    elif diagnosis == class2_dict[name]:
                        label_list.append(0)
                        
                labels.append(label_list)
            
            else:
                labels.append(1 if row["diagnosis"] == class1_dict[name] else 0)
              
        # Add extracted values and labels to arrays for returning 
        dfs_values.append(values)             
        dfs_labels.append(labels)
        
        
    return dfs_values, dfs_labels

In [15]:
def get_data(dataset):
    # Filter out empty arrays
    df = df_dict[dataset]
    if not multiple_labels_dict[dataset]:
        df = df[df["recording_duration"] > 10]        
    
    df_train_all, df_test, df_train, df_val = train_val_test_split2(df, dataset)       

    
    #SHUFFLE
    df_train_all = df_train_all.sample(frac=1).reset_index(drop=True)
    df_train = df_train.sample(frac=1).reset_index(drop=True)
    df_test = df_test.sample(frac=1).reset_index(drop=True)
    df_val = df_val.sample(frac=1).reset_index(drop=True)
    
    # Extract data values and labels from individual datasets
    dfs = [df_train_all, df_test, df_train, df_val]
    dfs_values, dfs_labels = data_to_values_and_labels(dfs, dataset) 

    #       Extract all values and labels from their respective datasets
    train_all_values, train_all_labels = dfs_values[0], dfs_labels[0]
    test_values, test_labels = dfs_values[1], dfs_labels[1]
    train_values, train_labels = dfs_values[2], dfs_labels[2]
    val_values, val_labels = dfs_values[3], dfs_labels[3]
    
    
    train_all_values, train_all_labels, test_values, test_labels, train_values, train_labels, val_values, val_labels = np.asarray(train_all_values), np.asarray(train_all_labels), np.asarray(test_values), np.asarray(test_labels), np.asarray(train_values), np.asarray(train_labels), np.asarray(val_values), np.asarray(val_labels)
    return [train_all_values, train_all_labels, test_values, test_labels, train_values, train_labels, val_values, val_labels]        

In [16]:
class EEG_Data_Generator_Heterogeneous(keras.utils.Sequence):
    
    def __init__(self, file_paths, file_labels, batch_size, dataset, conv=False, class_filter = None):  
        
        # Store file paths and corresponding labels
        self.file_paths = file_paths
        self.file_names = [file_path.split("/")[-1] for file_path in file_paths]
        self.file_labels = file_labels
        self.dataset = dataset
        df = df_dict[dataset]
        
        #Store class filter and class counters
        self.class_filter = class_filter
        self.class0_counter = 0
        self.class1_counter = 0
        
        # Store batch size
        self.batch_size = batch_size
        
        #initialize reshape stuff
        self.conv = conv
        
        # Calculate total lentgh of the data
        df_gen = df[df["file_name"].isin(self.file_names)]
        df_gen = df_gen.set_index("file_name")
        df_gen = df_gen.loc[self.file_names]
    
        if multiple_labels_dict[self.dataset]:
            self.number_of_windows = window_list(df_gen["number_of_windows"].tolist())
            self.total_windows = total_windows(self.number_of_windows)
        else:            
            self.windows = np.floor((df_gen["recording_duration"] - 1) / 10).astype(int)
            self.windows_cumsum = self.windows.cumsum()
            self.total_windows = self.windows_cumsum.iloc[-1]

        # Make heterogeneously randomized list of tuples of file index and sample index in that file
        if multiple_labels_dict[self.dataset]:
            j = 0
            file_and_sample_index = []
            file_and_sample_label = []
            for file_index in range(len(self.file_paths)):
                file_labels = self.file_labels[file_index]
                file_paths = []
                for i in range(len(file_labels)):
                    file_path = self.file_paths[file_index]+'_'+str(i)
                    file_paths.append(file_path)
                    for sample_index in range(self.number_of_windows[file_index][i]):
                        file_and_sample_index.append((file_index, i, sample_index))
                        file_and_sample_label.append(file_labels[i])
                        
                        if file_labels[i] == 0 and self.class0_counter < 10000:
                            file_and_sample_index.append((file_index, i, sample_index))
                            file_and_sample_label.append(file_labels[i])
                            self.class0_counter += 1
                        elif file_labels[i] == 1 and self.class1_counter < 10000:
                            file_and_sample_index.append((file_index, i, sample_index))
                            file_and_sample_label.append(file_labels[i])
                            self.class1_counter += 1
                        
        
        else:
            j = 0
            file_and_sample_index = []
            file_and_sample_label = []
            for file_index in range(len(self.file_paths)):
                file_path = self.file_paths[file_index]
                file_label = self.file_labels[file_index]
                for sample_index in range(self.windows[file_index]):                            
                    if file_label == 0 and self.class0_counter < 10000:
                        file_and_sample_index.append((file_index, sample_index))
                        file_and_sample_label.append(file_label)
                        self.class0_counter += 1
                    elif file_label == 1 and self.class1_counter < 10000:
                        file_and_sample_index.append((file_index, sample_index))
                        file_and_sample_label.append(file_label)
                        self.class1_counter += 1
                       
                    
                    
                    
        #Shuffle data
        split = list(zip(file_and_sample_index, file_and_sample_label))

        random.shuffle(split)

        file_and_sample_index, file_and_sample_label = list(zip(*split))

        self.split_files_and_samples = []
        self.split_labels = []
        length = int(np.floor(len(file_and_sample_label)/batch_size))
        
        for i in range(length):
            start_index = batch_size*i
            end_index = batch_size*(i+1)
            self.split_files_and_samples.append(np.asarray(file_and_sample_index[start_index:end_index]))
            self.split_labels.append(np.asarray(file_and_sample_label[start_index:end_index]))
            
       
        
    def __len__(self):
        return (np.ceil((self.class0_counter + self.class1_counter) / float(self.batch_size))-1).astype(np.int)           

    def get_class_weights(self):
        total = self.class0_counter + self.class1_counter
        class0_weight = self.class1_counter/total
        class1_weight = self.class0_counter/total
        return {0 : class0_weight, 1 : class1_weight}
    def __getitem__(self, idx):
        
        # Define function to append values and labels to the return values and labels
        def append_values_and_labels(batch_values, file_data_h5, file_path, sample_index, index=None):
            file_name = file_path.split("/")[-1][:-4]
            if plus_edf_dict[self.dataset]:
                file_name = file_name + '.edf'
            
            #Get file values
            if index is None:
                file_values = file_data_h5[file_name][sample_index]   
            else:
                file_values = file_data_h5[file_name+"_"+str(index)][sample_index]   
            
            if self.conv:
                file_values = np.swapaxes(file_values, 0, 1)
            else:
                file_values = file_values.reshape((1, file_values.shape[0], file_values.shape[1]))
            batch_values.append(file_values)

            return batch_values
        
        # Read in values for all file paths and duplicate file labels according the the amount of values
        #       Select subset of data
        batch_files_and_samples = self.split_files_and_samples[idx]
        batch_labels = self.split_labels[idx]
        
        #       Open raw data file
        h5f = h5py.File(h5f_dict[self.dataset], 'r')

        #       Get values for filepaths in batch
        batch_values = []
        
        if  multiple_labels_dict[self.dataset]:
            for file_path_index, index, sample_index in batch_files_and_samples:
                file_path = self.file_paths[file_path_index]
                batch_values = append_values_and_labels(batch_values, file_data_h5=h5f, file_path=file_path, sample_index=sample_index, index = index)
        else:
            for file_path_index, sample_index in batch_files_and_samples:
                file_path = self.file_paths[file_path_index]
                batch_values = append_values_and_labels(batch_values, file_data_h5=h5f, file_path=file_path, sample_index=sample_index)
        
        h5f.close()
        self.p = np.random.permutation(len(batch_values))
        batch_values, batch_labels = np.asarray(batch_values)[self.p], np.asarray(batch_labels)[self.p]
        return batch_values, batch_labels

In [17]:
def number_class(df, dataset):
    number_class0 = 0
    number_class1 = 0
    
    class0 = class1_dict[dataset]
    class1 = class2_dict[dataset]
    for index, row in df.iterrows():
        if not multiple_labels_dict[dataset]:
            if class1_dict[dataset] == row["diagnosis"]:
                number_class0 += np.floor((row["recording_duration"] - 1) / 10).astype(int)
            else:
                number_class1 += np.floor((row["recording_duration"] - 1) / 10).astype(int)
        else:
            diagnoses = row["diagnosis"]
            diagnoses = diagnoses.split(" ")
            diagnoses.remove('')

            number_of_windows = row["number_of_windows"]
            number_of_windows = number_of_windows.split(" ")
            number_of_windows.remove('')
            for i in range(len(diagnoses)):
                diagnosis = diagnoses[i]
                windows = number_of_windows[i]
                if diagnosis == class0:
                    number_class0 += int(windows)
                elif diagnosis == class1:
                    number_class1 += int(windows)
    return int(number_class0), int(number_class1)

In [18]:
#
#   SPLIT DATA IN ALMOST EQUAL PARTS
#
def stratify_ensemble_split(df, dataset, split_size):
    #shuffle randomly
    df = df.sample(frac=1).reset_index(drop=True)
    
    # Get patient ids
    patient_ids = np.unique(df["patient_id"])

    # Get (cumulative) number of tokens per unique patient
    patient_tokens_cumsum = get_tokens_cumsum(df, dataset)
    
    # take approx split size subset of df
    split_index = next(index for index, curr_patient_tokens_cumsum in enumerate(patient_tokens_cumsum) if curr_patient_tokens_cumsum > split_size) + 1
    stratified_ids = patient_ids[:split_index]
    df = df[df["patient_id"].isin(stratified_ids)]
    
    return df

## Model performance functions

In [19]:
#
#   HELP VISUALIZE LEARNING PROGRESSION
#
def plot_history(history):
    plt.figure(figsize = (12,16))
    plt.subplot(4,2,1)

    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    try:
        plt.plot(history.epoch, np.array(history.history['acc']),'g-',
               label='Train accuracy')
        plt.plot(history.epoch, np.array(history.history['val_acc']),'r-',
               label = 'Validation accuracy')
    except:
        plt.plot(history.epoch, np.array(history.history['accuracy_m']),'g-',
               label='Train accuracy')
        plt.plot(history.epoch, np.array(history.history['val_accuracy_m']),'r-',
               label = 'Validation accuracy')
    plt.ylim([0.0,1.0])
    plt.legend()

    plt.subplot(4,2,2)
    plt.xlabel('Epoch')
    plt.ylabel('Loss minimised by model')
    plt.plot(history.epoch, np.array(history.history['loss']),'g-',
           label='Train loss')
    plt.plot(history.epoch, np.array(history.history['val_loss']),'r-',
           label = 'Validation loss')
    plt.legend()
    
    plt.subplot(4,2,3)
    plt.xlabel('Epoch')
    plt.ylabel('Recall')
    plt.plot(history.epoch, np.array(history.history['recall_m']),'g-',
           label='Train recall')
    plt.plot(history.epoch, np.array(history.history['val_recall_m']),'r-',
           label = 'Validation recall')
    plt.legend()
    
    plt.subplot(4,2,4)
    plt.xlabel('Epoch')
    plt.ylabel('Negative recall')
    plt.plot(history.epoch, np.array(history.history['neg_recall_m']),'g-',
           label='Train neg. recall')
    plt.plot(history.epoch, np.array(history.history['val_neg_recall_m']),'r-',
           label = 'Validation neg. recall')
    plt.legend()
    
    plt.subplot(4,2,5)
    plt.xlabel('Epoch')
    plt.ylabel('Precision')
    plt.plot(history.epoch, np.array(history.history['precision_m']),'g-',
           label='Train precision')
    plt.plot(history.epoch, np.array(history.history['val_precision_m']),'r-',
           label = 'Validation precision')
    plt.legend()
    
    plt.subplot(4,2,6)
    plt.xlabel('Epoch')
    plt.ylabel('Negative precision')
    plt.plot(history.epoch, np.array(history.history['neg_precision_m']),'g-',
           label='Train neg. precision')
    plt.plot(history.epoch, np.array(history.history['val_neg_precision_m']),'r-',
           label = 'Validation neg. precision')
    plt.legend()
    
    plt.subplot(4,2,7)
    plt.xlabel('Epoch')
    plt.ylabel('F1')
    plt.plot(history.epoch, np.array(history.history['f1_m']),'g-',
           label='Train F1')
    plt.plot(history.epoch, np.array(history.history['val_f1_m']),'r-',
           label = 'Validation F1')
    plt.legend()
    
    plt.subplot(4,2,8)
    plt.xlabel('Epoch')
    plt.ylabel('Negative F1')
    plt.plot(history.epoch, np.array(history.history['neg_f1_m']),'g-',
           label='Train neg. F1')
    plt.plot(history.epoch, np.array(history.history['val_neg_f1_m']),'r-',
           label = 'Validation neg. F1')
    plt.legend()
    
    

In [20]:
#
#   CALCULATE RECALL, PRECISION, AND F1-SCORE PERFORMANCE MEASURES
#
#w = 3.2552225249772935
def w_binary_crossentropy(y_true, y_pred):
    weights = y_true * class1_weight + (1. - y_true) * class0_weight
    bce = keras.backend.binary_crossentropy(y_true, y_pred)
    weighted_bce = keras.backend.mean(bce * weights)
    return weighted_bce


def tp_m(y_true, y_pred):
    tp = keras.backend.sum(keras.backend.round(keras.backend.clip(y_true * y_pred, 0, 1)))
    return tp

def fp_m(y_true, y_pred):
    fp = keras.backend.sum(keras.backend.round(keras.backend.clip((1-y_true) * y_pred, 0, 1)))
    return fp

def fn_m(y_true, y_pred):
    fn = keras.backend.sum(keras.backend.round(keras.backend.clip(y_true * (1-y_pred), 0, 1)))
    return fn

def tn_m(y_true, y_pred):
    tn = keras.backend.sum(keras.backend.round(keras.backend.clip((1-y_true) * (1-y_pred), 0, 1)))
    return tn

def accuracy_m(y_true, y_pred):
    accuracy = (tp_m(y_true, y_pred) + tn_m(y_true, y_pred)) / (tp_m(y_true, y_pred) + fp_m(y_true, y_pred) + tn_m(y_true, y_pred) + fn_m(y_true, y_pred) + keras.backend.epsilon())
    return accuracy

def pos_true_m(y_true, y_pred):
    return tp_m(y_true, y_pred) + fn_m(y_true, y_pred) 

def pos_pred_m(y_true, y_pred):
    return tp_m(y_true, y_pred) + fp_m(y_true, y_pred)

def neg_true_m(y_true, y_pred):
    return tn_m(y_true, y_pred) + fp_m(y_true, y_pred)

def neg_pred_m(y_true, y_pred):
    return tn_m(y_true, y_pred) + fn_m(y_true, y_pred)

# Sensitivity
def recall_m(y_true, y_pred):
    recall = tp_m(y_true, y_pred) / (tp_m(y_true, y_pred) + fn_m(y_true, y_pred) + keras.backend.epsilon())
    return recall

# Specificity
def neg_recall_m(y_true, y_pred):
    neg_recall = tn_m(y_true, y_pred) / (tn_m(y_true, y_pred) + fp_m(y_true, y_pred) + keras.backend.epsilon())
    return neg_recall

def precision_m(y_true, y_pred):
    precision = tp_m(y_true, y_pred) / (tp_m(y_true, y_pred) + fp_m(y_true, y_pred) + keras.backend.epsilon())
    return precision

# Negative predictive value
def neg_precision_m(y_true, y_pred):
    neg_precision = tn_m(y_true, y_pred) / (tn_m(y_true, y_pred) + fn_m(y_true, y_pred) + keras.backend.epsilon())
    return neg_precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+keras.backend.epsilon()))

def neg_f1_m(y_true, y_pred):
    neg_precision = neg_precision_m(y_true, y_pred)
    neg_recall = neg_recall_m(y_true, y_pred)
    return 2 * ((neg_precision * neg_recall) / (neg_precision + neg_recall + keras.backend.epsilon()))

def balanced_acc_m(y_true, y_pred):
    return (recall_m(y_true, y_pred) + neg_recall_m(y_true, y_pred))/2

In [21]:
def perf_measure(y_actual, y_hat):
    y_actual = np.asarray(y_actual)
    y_hat = np.asarray(y_hat)
    
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    
    for i in range(len(y_hat)): 
        if len(y_actual.shape) == 2 and y_actual.shape[1] == 2:
            y_actual_ = np.argmax(y_actual[i])
        else:
            y_actual_ = y_actual[i]
        
        if len(y_hat.shape) == 2 and y_hat.shape[1] == 2:
            y_hat_ = np.argmax(y_hat[i])
        else:
            y_hat_ = y_hat[i]
            
        if y_actual_==y_hat_==1:
            TP += 1
        if y_hat_==1 and y_actual_!=y_hat_:
            FP += 1
        if y_actual_==y_hat_==0:
            TN += 1
        if y_hat_==0 and y_actual_!=y_hat_:
            FN += 1
    
    print(f"tp({TP}), fp({FP}), tn({TN}), fn({FN})")
    
    accuracy = (TP + TN) / (TN + TP + FP + FN)
    recall = TP / (FN + TP)
    neg_recall = TN / (FP + TN)
    
    try:
        precision = TP / (FP + TP)
    except:
        print("No positive predictions...")
        precision = 0
        
    try:
        neg_precision = TN / (FN + TN)
    except:
        print("No negative predictions...")
        neg_precision = 0
            
    try:
        f1 = 2 * (recall * precision) / (recall + precision)
    except:
        f1 = 0
        
    try:
        neg_f1 = 2 * (neg_recall * neg_precision) / (neg_recall + neg_precision)
    except:
        neg_f1 = 0

    aupr = average_precision_score(np.asarray(y_actual), np.asarray(y_hat))
    neg_aupr = average_precision_score(np.asarray([1 - y_a for y_a in y_actual]), np.asarray([1 - y_h for y_h in y_hat]))

    print(f"acc={accuracy}, recall={recall}, neg_recall={neg_recall}, precision={precision}, neg_precision={neg_precision}, f1={f1}, neg_f1={neg_f1}, aupr={aupr}, neg_aupr={neg_aupr}")
    
    return accuracy, recall, neg_recall, precision, neg_precision, f1, neg_f1, aupr, neg_aupr

## Class imbalance

In [22]:
def class_weights(dataset, file_paths, file_labels):
    #number of windows of label, number of windows of window 
    class0 = 0
    class1 = 0
    
    # Store file paths and corresponding labels
    file_names = [file_path.split("/")[-1] for file_path in file_paths]
    df = df_dict[dataset]
        
    # Calculate total lentgh of the data
    df_gen = df[df["file_name"].isin(file_names)]
    df_gen = df_gen.set_index("file_name")
    df_gen = df_gen.loc[file_names]
    
    #calc number of windows
    if multiple_labels_dict[dataset]:
        number_of_windows = window_list(df_gen["number_of_windows"].tolist())
    else:            
        windows = np.floor((df_gen["recording_duration"] - 1) / 10).astype(int)
        
   # Make heterogeneously randomized list of tuples of file index and sample index in that file
    if multiple_labels_dict[dataset]:
        for file_index in range(len(file_paths)):
            this_file_labels = file_labels[file_index]
            for i in range(len(this_file_labels)):
                if this_file_labels[i] == 0:
                    class0 += number_of_windows[file_index][i]
                else:
                    class1 += number_of_windows[file_index][i]
                        
        
    else:
        for file_index in range(len(file_paths)):
            file_label = file_labels[file_index]
            if file_label == 0:
                class0 += windows[file_index]
            else:
                class1 += windows[file_index]    
    
    total = class0 + class1
    class0_weight = class1/total
    class1_weight = class0/total
    return {0 : class0_weight, 1 : class1_weight}
    

In [23]:
h5f = h5py.File(h5f_dict["TUEP"], 'r')
#print(h5f.keys())

### EEGNET

In [24]:
def train_val_test_split2(df, dataset):
     # Get patient ids
    patient_ids = np.unique(df["patient_id"])
    #if dataset == "TUEP":
     #   random.Random(5).shuffle(patient_ids)
        #patient_ids = np.concatenate((patient_ids[:38], patient_ids[111:]))
    # Get (cumulative) number of tokens per unique epilepsy patient
    patient_tokens_cumsum = get_tokens_cumsum(df, dataset, patient_ids)
    total_tokens = np.amax(patient_tokens_cumsum)

    # Split patient_ids based on number of recordings; 80% of all tokens should be training data
    # (of which 64% training data and 16% validation data) and 20% should be test data.
    #       Find patient_index corresponding to a cumulative 80% of the data to split training from test data      
    test_split = next(index for index, curr_patient_tokens_cumsum in enumerate(patient_tokens_cumsum) if curr_patient_tokens_cumsum > 0.8 * total_tokens) + 1
        
    
    #       Use split patient_id to split off 20% of the data for test data
    train_all_patient_ids = patient_ids[:test_split]
    train_all_patient_tokens = patient_tokens_cumsum[:test_split]
    
    test_patient_ids = patient_ids[test_split:]
    test_patient_tokens = patient_tokens_cumsum[test_split:]

    #       Find patient_id corresponding to a cumulative 80% of the training data to split training data from validation data
    total_train_tokens = np.amax(train_all_patient_tokens)
    random.Random(2).shuffle(train_all_patient_ids)
    train_all_patient_tokens = get_tokens_cumsum(df, dataset,train_all_patient_ids)
    val_split = next(index for index, curr_patient_tokens_cumsum in enumerate(train_all_patient_tokens) if curr_patient_tokens_cumsum > 0.8 * total_train_tokens) + 1

    #       Use split patient_id to split off 20% of the data for validation data
    train_patient_ids = train_all_patient_ids[:val_split]
    train_patient_tokens = train_all_patient_tokens[:val_split]

    val_patient_ids = train_all_patient_ids[val_split:]
    val_patient_tokens = train_all_patient_tokens[val_split:]
    print(val_split), print(test_split), print(len(patient_ids))
    
    # Split dataframe in train, validation and test sets based on the split made above
    df_train_all = df[df["patient_id"].isin(train_all_patient_ids)]
    df_test = df[df["patient_id"].isin(test_patient_ids)]
    
    df_train = df[df["patient_id"].isin(train_patient_ids)]
    df_val = df[df["patient_id"].isin(val_patient_ids)]
    
    return df_train_all, df_test, df_train, df_val   

In [25]:
def get_eegnet0():
    
    F1 = 4
    C = 19
    D = 2
    F2 = D * F1

    do = [0.5, 0.25, 0]
    
    
    model = Sequential()
    model.add(Dropout(do[0],input_shape=(1, C, 2560)))
    model.add(Conv2D(filters=F1, kernel_size=(1, 128), input_shape=(1, C, 2560), padding="same"))
    model.add(Activation("linear"))
    model.add(BatchNormalization(axis=1))

    model.add(DepthwiseConv2D(kernel_size=(C, 1), padding="valid", depth_multiplier=D, data_format='channels_first'))
    model.add(Activation("linear"))
    model.add(BatchNormalization(axis=1))
    model.add(Activation("elu"))

    model.add(AveragePooling2D(pool_size=(1, 4), data_format='channels_first'))
    model.add(Dropout(do[1]))

    model.add(SeparableConv2D(filters=F2, kernel_size=(1, 16), padding="same", use_bias=False))
    model.add(Activation("linear"))
    
    model.add(BatchNormalization(axis=1))
    model.add(Activation("elu"))

    model.add(AveragePooling2D(pool_size=(1, 8), data_format='channels_first'))
    model.add(Dropout(do[2]))


    model.add(Flatten())
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    learning_rate = 0.001
    
    
    model.compile(loss= 'binary_crossentropy',
                  optimizer=keras.optimizers.Adam(learning_rate),
                  metrics=[balanced_acc_m, pos_pred_m, neg_pred_m, tp_m, fp_m, tn_m, fn_m, recall_m, neg_recall_m, precision_m, neg_precision_m])
    
    return model

In [26]:
def get_eegnet1():
    
    F1 = 4
    C = 19
    D = 2
    F2 = D * F1

    do = [0.5, 0.25, 0]
    
    
    model = Sequential()
    model.add(Dropout(do[0],input_shape=(1, C, 2560)))
    model.add(Conv2D(filters=F1, kernel_size=(1, 128), input_shape=(1, C, 2560), padding="same"))
    model.add(Activation("linear"))
    model.add(BatchNormalization(axis=1))

    model.add(DepthwiseConv2D(kernel_size=(C, 1), padding="valid", depth_multiplier=D, data_format='channels_first'))
    model.add(Activation("linear"))
    model.add(BatchNormalization(axis=1))
    model.add(Activation("elu"))

    model.add(AveragePooling2D(pool_size=(1, 4), data_format='channels_first'))
    model.add(Dropout(do[1]))

    model.add(SeparableConv2D(filters=F2, kernel_size=(1, 16), padding="same", use_bias=False))
    model.add(Activation("linear"))
    
    model.add(BatchNormalization(axis=1))
    model.add(Activation("elu"))

    model.add(AveragePooling2D(pool_size=(1, 8), data_format='channels_first'))
    model.add(Dropout(do[2]))


    model.add(Flatten())
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    learning_rate = 0.001
    
    
    model.compile(loss= 'binary_crossentropy',
                  optimizer=keras.optimizers.Adam(learning_rate),
                  metrics=[balanced_acc_m, pos_pred_m, neg_pred_m, tp_m, fp_m, tn_m, fn_m, recall_m, neg_recall_m, precision_m, neg_precision_m])
    
    return model

In [27]:
def clone_weights(model0, model1):
    nr_layers = len(my_model0.layers)
    i = 0
    for layer in model1.layers:
        layer.trainable = True #Just to be sure
        if i < nr_layers-15 and not layer.__class__.__name__ == "Dropout":
            layer.set_weights(model0.layers[i].get_weights())
        i += 1

In [28]:
import csv
f = open("/home/jupyter/time_series_transfer_learning/transfer_learning/9-multi-source-transfer-learning/logbooks/logbook_TUAB_TUSZ_source_val.csv", 'a+', newline='')
writer = csv.writer(f, lineterminator='\n')
writer.writerow(["Epoch", "bal_acc_0", "bal_acc_1", "bal_acc_avg"])
f.close()

In [ ]:
keras.backend.clear_session()
#source model
source_model0 = "TUAB"
source_model1 = "TUSZ"
target_model = "TUEP"

my_model0, my_model1 = get_eegnet0(), get_eegnet1()
batch_size = 256



#Generator
train_all_values, train_all_labels, test_values, test_labels, train_values, train_labels, val_values, val_labels = get_data(source_model0)
train_generator0 = EEG_Data_Generator_Heterogeneous(train_values, train_labels, batch_size, source_model0, class_filter = None)
val_generator0 = EEG_Data_Generator_Heterogeneous(val_values, val_labels, batch_size, source_model0, class_filter = None)

train_all_values, train_all_labels, test_values, test_labels, train_values, train_labels, val_values, val_labels = get_data(source_model1)
train_generator1 = EEG_Data_Generator_Heterogeneous(train_values, train_labels, batch_size, source_model1, class_filter = None)
val_generator1 = EEG_Data_Generator_Heterogeneous(val_values, val_labels, batch_size, source_model1, class_filter = None)

#Training process
for epoch in range(150):
    print("Epoch: " + str(epoch))
    #Train on source_model0
    my_model0.fit(train_generator0, verbose = 0)
    
    #Update weights of source_model1
    clone_weights(my_model0, my_model1)
        
    #Train on source_model1
    my_model1.fit(train_generator1, verbose = 0)
        
    #Update weights of source_model1
    clone_weights(my_model1, my_model0)
    
    #Validate this epoch
    val_dict0 = my_model0.evaluate(val_generator0, steps=len(val_generator0), return_dict = True, verbose = 0)
    val_dict1 = my_model1.evaluate(val_generator1, steps=len(val_generator1), return_dict = True, verbose = 0)
    
    #Write the results in a logbook
    f = open("/home/jupyter/time_series_transfer_learning/transfer_learning/9-multi-source-transfer-learning/logbooks/logbook_TUAB_TUSZ_source_val.csv", 'a+', newline='')
    writer = csv.writer(f, lineterminator='\n')
    writer.writerow([epoch, val_dict0['balanced_acc_m'], val_dict1['balanced_acc_m'], (val_dict0['balanced_acc_m']+val_dict1['balanced_acc_m'])/2])
    f.close()
    #Save weights of the model
    my_model0.save("/home/jupyter/time_series_transfer_learning/transfer_learning/9-multi-source-transfer-learning/logbooks/model_weights/TUAB_TUSZ_source_val_epoch_" + str(epoch))    

725
907
1119
54
71
88


In [ ]:
import csv
f = open("/home/jupyter/time_series_transfer_learning/transfer_learning/9-multi-source-transfer-learning/logbooks/logbook_TUEP_TUSZ_source_val.csv", 'a+', newline='')
writer = csv.writer(f, lineterminator='\n')
writer.writerow(["Epoch", "bal_acc_0", "bal_acc_1", "bal_acc_avg"])
f.close()

keras.backend.clear_session()
#source model
source_model0 = "TUEP"
source_model1 = "TUSZ"
target_model = "TUAB"

my_model0, my_model1 = get_eegnet0(), get_eegnet1()
batch_size = 256



#Generator
train_all_values, train_all_labels, test_values, test_labels, train_values, train_labels, val_values, val_labels = get_data(source_model0)
train_generator0 = EEG_Data_Generator_Heterogeneous(train_values, train_labels, batch_size, source_model0, class_filter = None)
val_generator0 = EEG_Data_Generator_Heterogeneous(val_values, val_labels, batch_size, source_model0, class_filter = None)

train_all_values, train_all_labels, test_values, test_labels, train_values, train_labels, val_values, val_labels = get_data(source_model1)
train_generator1 = EEG_Data_Generator_Heterogeneous(train_values, train_labels, batch_size, source_model1, class_filter = None)
val_generator1 = EEG_Data_Generator_Heterogeneous(val_values, val_labels, batch_size, source_model1, class_filter = None)

#Training process
for epoch in range(150):
    print("Epoch: " + str(epoch))
    #Train on source_model0
    my_model0.fit(train_generator0, verbose = 0)
    
    #Update weights of source_model1
    clone_weights(my_model0, my_model1)
        
    #Train on source_model1
    my_model1.fit(train_generator1, verbose = 0)
        
    #Update weights of source_model1
    clone_weights(my_model1, my_model0)
    
    #Validate this epoch
    val_dict0 = my_model0.evaluate(val_generator0, steps=len(val_generator0), return_dict = True, verbose = 0)
    val_dict1 = my_model1.evaluate(val_generator1, steps=len(val_generator1), return_dict = True, verbose = 0)
    
    #Write the results in a logbook
    f = open("/home/jupyter/time_series_transfer_learning/transfer_learning/9-multi-source-transfer-learning/logbooks/logbook_TUEP_TUSZ_source_val.csv", 'a+', newline='')
    writer = csv.writer(f, lineterminator='\n')
    writer.writerow([epoch, val_dict0['balanced_acc_m'], val_dict1['balanced_acc_m'], (val_dict0['balanced_acc_m']+val_dict1['balanced_acc_m'])/2])
    f.close()
    #Save weights of the model
    my_model0.save("/home/jupyter/time_series_transfer_learning/transfer_learning/9-multi-source-transfer-learning/logbooks/model_weights/TUEP_TUSZ_source_val_epoch_" + str(epoch))  

In [ ]:
import csv
f = open("/home/jupyter/time_series_transfer_learning/transfer_learning/9-multi-source-transfer-learning/logbooks/logbook_TUEP_TUAB_source_val.csv", 'a+', newline='')
writer = csv.writer(f, lineterminator='\n')
writer.writerow(["Epoch", "bal_acc_0", "bal_acc_1", "bal_acc_avg"])
f.close()

keras.backend.clear_session()
#source model
source_model0 = "TUEP"
source_model1 = "TUAB"
target_model = "TUSZ"

my_model0, my_model1 = get_eegnet0(), get_eegnet1()
batch_size = 256



#Generator
train_all_values, train_all_labels, test_values, test_labels, train_values, train_labels, val_values, val_labels = get_data(source_model0)
train_generator0 = EEG_Data_Generator_Heterogeneous(train_values, train_labels, batch_size, source_model0, class_filter = None)
val_generator0 = EEG_Data_Generator_Heterogeneous(val_values, val_labels, batch_size, source_model0, class_filter = None)

train_all_values, train_all_labels, test_values, test_labels, train_values, train_labels, val_values, val_labels = get_data(source_model1)
train_generator1 = EEG_Data_Generator_Heterogeneous(train_values, train_labels, batch_size, source_model1, class_filter = None)
val_generator1 = EEG_Data_Generator_Heterogeneous(val_values, val_labels, batch_size, source_model1, class_filter = None)

#Training process
for epoch in range(150):
    print("Epoch: " + str(epoch))
    #Train on source_model0
    my_model0.fit(train_generator0, verbose = 0)
    
    #Update weights of source_model1
    clone_weights(my_model0, my_model1)
        
    #Train on source_model1
    my_model1.fit(train_generator1, verbose = 0)
        
    #Update weights of source_model1
    clone_weights(my_model1, my_model0)
    
    #Validate this epoch
    val_dict0 = my_model0.evaluate(val_generator0, steps=len(val_generator0), return_dict = True, verbose = 0)
    val_dict1 = my_model1.evaluate(val_generator1, steps=len(val_generator1), return_dict = True, verbose = 0)
    
    #Write the results in a logbook
    f = open("/home/jupyter/time_series_transfer_learning/transfer_learning/9-multi-source-transfer-learning/logbooks/logbook_TUEP_TUAB_source_val.csv", 'a+', newline='')
    writer = csv.writer(f, lineterminator='\n')
    writer.writerow([epoch, val_dict0['balanced_acc_m'], val_dict1['balanced_acc_m'], (val_dict0['balanced_acc_m']+val_dict1['balanced_acc_m'])/2])
    f.close()
    #Save weights of the model
    my_model0.save("/home/jupyter/time_series_transfer_learning/transfer_learning/9-multi-source-transfer-learning/logbooks/model_weights/TUEP_TUAB_source_val_epoch_" + str(epoch)) 

In [55]:
def get_eegnet():
    
    F1 = 4
    C = 19
    D = 2
    F2 = D * F1

    do = [0, 0.25, 0]
    
    
    model = Sequential()
    model.add(Dropout(do[0],input_shape=(1, C, 2560)))
    model.add(Conv2D(filters=F1, kernel_size=(1, 128), input_shape=(1, C, 2560), padding="same"))
    model.add(Activation("linear"))
    model.add(BatchNormalization(axis=1))

    model.add(DepthwiseConv2D(kernel_size=(C, 1), padding="valid", depth_multiplier=D, data_format='channels_first'))
    model.add(Activation("linear"))
    model.add(BatchNormalization(axis=1))
    model.add(Activation("elu"))

    model.add(AveragePooling2D(pool_size=(1, 4), data_format='channels_first'))
    model.add(Dropout(do[1]))

    model.add(SeparableConv2D(filters=F2, kernel_size=(1, 16), padding="same", use_bias=False))
    model.add(Activation("linear"))
    
    model.add(BatchNormalization(axis=1))
    model.add(Activation("elu"))

    model.add(AveragePooling2D(pool_size=(1, 8), data_format='channels_first'))
    model.add(Dropout(do[2]))


    model.add(Flatten())
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    learning_rate = 0.001
    
    
    model.compile(loss= 'binary_crossentropy',
                  optimizer=keras.optimizers.Adam(learning_rate),
                  metrics=[balanced_acc_m, pos_pred_m, neg_pred_m, tp_m, fp_m, tn_m, fn_m, recall_m, neg_recall_m, precision_m, neg_precision_m])
    
    return model

In [ ]:
#source model
basepath = "/home/jupyter/time_series_transfer_learning/transfer_learning/9-multi-source-transfer-learning/logbooks/model_weights/"
source_model = "TUAB_TUSZ"
target_model = "TUEP"
clone = tf.keras.models.load_model(basepath + source_model + "_source_val_epoch_" + str(149), custom_objects={'balanced_acc_m':balanced_acc_m, 'pos_pred_m': pos_pred_m, 'neg_pred_m': pos_pred_m, 'tp_m': tp_m, 'fp_m': fp_m, 'tn_m': tn_m, 'fn_m': fn_m, 'recall_m': recall_m, 'neg_recall_m': neg_recall_m, 'precision_m': precision_m, 'neg_precision_m': neg_precision_m})
my_model = get_eegnet()

#freeze layers
nr_layers = len(my_model.layers)
i = 0
for layer in my_model.layers:
    layer.trainable = True
    if i < nr_layers-15 and not layer.__class__.__name__ == "Dropout":
        layer.set_weights(clone.layers[i].get_weights())
        layer.trainable = False
    i += 1
    
keras.backend.clear_session()
dataset = target_model
batch_size = 256

train_all_values, train_all_labels, test_values, test_labels, train_values, train_labels, val_values, val_labels = get_data(dataset)

#Generator

train_generator = EEG_Data_Generator_Heterogeneous(train_all_values, train_all_labels, batch_size, dataset, class_filter = None)
val_generator = EEG_Data_Generator_Heterogeneous(val_values, val_labels, batch_size, dataset, class_filter = None)

#Weights
class_weights_dict = train_generator.get_class_weights()

epochs = 20
checkpoint_path = '/home/jupyter/time_series_transfer_learning/transfer_learning/9-multi-source-transfer-learning/logbooks/model_weights/tuab_tusz_tuep_train_all.hdf5'
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 monitor='val_balanced_acc_m',
                                                 save_weights_only=True,
                                                 verbose=1)
log_callback = tf.keras.callbacks.CSVLogger(filename =  '/home/jupyter/time_series_transfer_learning/transfer_learning/9-multi-source-transfer-learning/logbooks/logbook_tuab_tusz_tuep_train_all.csv', separator=",", append=True)
active_callbacks = [cp_callback, log_callback]


my_model_history = my_model.fit_generator(generator=train_generator,
                                          steps_per_epoch=len(train_generator),
                                          epochs=epochs,
                                          shuffle=True,
                                          #validation_data=val_generator,
                                          #validation_steps=len(val_generator),
                                          class_weight= class_weights_dict,
                                        callbacks = active_callbacks)

106
149
200
Epoch 1/20
78/78 [==============================] - ETA: 0s - loss: 0.3473 - balanced_acc_m: 0.5383 - pos_pred_m: 72.6154 - neg_pred_m: 183.3846 - tp_m: 41.2308 - fp_m: 31.3846 - tn_m: 96.6026 - fn_m: 86.7821 - recall_m: 0.3218 - neg_recall_m: 0.7548 - precision_m: 0.5898 - neg_precision_m: 0.5252
Epoch 00001: saving model to /home/jupyter/time_series_transfer_learning/transfer_learning/9-multi-source-transfer-learning/logbooks/model_weights/tuab_tusz_tuep_train_all.hdf5
78/78 [==============================] - 67s 856ms/step - loss: 0.3473 - balanced_acc_m: 0.5383 - pos_pred_m: 72.6154 - neg_pred_m: 183.3846 - tp_m: 41.2308 - fp_m: 31.3846 - tn_m: 96.6026 - fn_m: 86.7821 - recall_m: 0.3218 - neg_recall_m: 0.7548 - precision_m: 0.5898 - neg_precision_m: 0.5252
Epoch 2/20
30/78 [==========>...................] - ETA: 30s - loss: 0.3441 - balanced_acc_m: 0.5494 - pos_pred_m: 59.9333 - neg_pred_m: 196.0667 - tp_m: 35.6000 - fp_m: 24.3333 - tn_m: 106.3333 - fn_m: 89.7333 - reca

In [ ]:
#source model
basepath = "/home/jupyter/time_series_transfer_learning/transfer_learning/9-multi-source-transfer-learning/logbooks/model_weights/"
source_model = "TUEP_TUAB"
target_model = "TUSZ"
clone = tf.keras.models.load_model(basepath + source_model + "_source_val_epoch_" + str(149), custom_objects={'balanced_acc_m':balanced_acc_m, 'pos_pred_m': pos_pred_m, 'neg_pred_m': pos_pred_m, 'tp_m': tp_m, 'fp_m': fp_m, 'tn_m': tn_m, 'fn_m': fn_m, 'recall_m': recall_m, 'neg_recall_m': neg_recall_m, 'precision_m': precision_m, 'neg_precision_m': neg_precision_m})
my_model = get_eegnet()

#freeze layers
nr_layers = len(my_model.layers)
i = 0
for layer in my_model.layers:
    layer.trainable = True
    if i < nr_layers-15 and not layer.__class__.__name__ == "Dropout":
        layer.set_weights(clone.layers[i].get_weights())
        layer.trainable = False
    i += 1
    
keras.backend.clear_session()
dataset = target_model
batch_size = 256

train_all_values, train_all_labels, test_values, test_labels, train_values, train_labels, val_values, val_labels = get_data(dataset)

#Generator

train_generator = EEG_Data_Generator_Heterogeneous(train_all_values, train_all_labels, batch_size, dataset, class_filter = None)
val_generator = EEG_Data_Generator_Heterogeneous(val_values, val_labels, batch_size, dataset, class_filter = None)

#Weights
class_weights_dict = train_generator.get_class_weights()

epochs = 150
checkpoint_path = '/home/jupyter/time_series_transfer_learning/transfer_learning/9-multi-source-transfer-learning/logbooks/model_weights/tuep_tuab_tusz_train_all.hdf5'
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 monitor='val_balanced_acc_m',
                                                 save_weights_only=True,
                                                 verbose=1)
log_callback = tf.keras.callbacks.CSVLogger(filename =  '/home/jupyter/time_series_transfer_learning/transfer_learning/9-multi-source-transfer-learning/logbooks/logbook_tuep_tuab_tusz_train_all.csv', separator=",", append=True)
active_callbacks = [cp_callback, log_callback]


my_model_history = my_model.fit_generator(generator=train_generator,
                                          steps_per_epoch=len(train_generator),
                                          epochs=epochs,
                                          shuffle=True,
                                          #validation_data=val_generator,
                                          #validation_steps=len(val_generator),
                                          class_weight= class_weights_dict,
                                        callbacks = active_callbacks)

In [ ]:
#source model
basepath = "/home/jupyter/time_series_transfer_learning/transfer_learning/9-multi-source-transfer-learning/logbooks/model_weights/"
source_model = "TUEP_TUSZ"
target_model = "TUAB"
clone = tf.keras.models.load_model(basepath + source_model + "_source_val_epoch_" + str(69), custom_objects={'balanced_acc_m':balanced_acc_m, 'pos_pred_m': pos_pred_m, 'neg_pred_m': pos_pred_m, 'tp_m': tp_m, 'fp_m': fp_m, 'tn_m': tn_m, 'fn_m': fn_m, 'recall_m': recall_m, 'neg_recall_m': neg_recall_m, 'precision_m': precision_m, 'neg_precision_m': neg_precision_m})
my_model = get_eegnet()

#freeze layers
nr_layers = len(my_model.layers)
i = 0
for layer in my_model.layers:
    layer.trainable = True
    if i < nr_layers-15 and not layer.__class__.__name__ == "Dropout":
        layer.set_weights(clone.layers[i].get_weights())
        layer.trainable = False
    i += 1
    
keras.backend.clear_session()
dataset = target_model
batch_size = 256

train_all_values, train_all_labels, test_values, test_labels, train_values, train_labels, val_values, val_labels = get_data(dataset)

#Generator

train_generator = EEG_Data_Generator_Heterogeneous(train_all_values, train_all_labels, batch_size, dataset, class_filter = None)
val_generator = EEG_Data_Generator_Heterogeneous(val_values, val_labels, batch_size, dataset, class_filter = None)

#Weights
class_weights_dict = train_generator.get_class_weights()

epochs = 20
checkpoint_path = '/home/jupyter/time_series_transfer_learning/transfer_learning/9-multi-source-transfer-learning/logbooks/model_weights/tuep_tusz_tuab_train_all.hdf5'
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 monitor='val_balanced_acc_m',
                                                 save_weights_only=True,
                                                 verbose=1)
log_callback = tf.keras.callbacks.CSVLogger(filename =  '/home/jupyter/time_series_transfer_learning/transfer_learning/9-multi-source-transfer-learning/logbooks/logbook_tuep_tusz_tuab_train_all.csv', separator=",", append=True)
active_callbacks = [cp_callback, log_callback]


my_model_history = my_model.fit_generator(generator=train_generator,
                                          steps_per_epoch=len(train_generator),
                                          epochs=epochs,
                                          shuffle=True,
                                          #validation_data=val_generator,
                                          #validation_steps=len(val_generator),
                                          class_weight= class_weights_dict,
                                        callbacks = active_callbacks)

In [64]:
#Load my_model
basepath = "/home/jupyter/time_series_transfer_learning/transfer_learning/9-multi-source-transfer-learning/logbooks/model_weights/"
path = basepath +  "tuab_tusz_tuep_train_all.hdf5"

my_model = get_eegnet()
my_model.load_weights(path)


#Make test generator
dataset = "TUEP"
train_all_values, train_all_labels, test_values, test_labels, train_values, train_labels, val_values, val_labels = get_data(dataset)
test_generator = EEG_Data_Generator_Heterogeneous(test_values, test_labels, batch_size, dataset)

#Evaluate
test_dict = my_model.evaluate(test_generator, steps=len(test_generator), return_dict = True)
test_preds = my_model.predict_generator(test_generator, steps=len(test_generator))
test_labels_ = np.concatenate([test_generator[i][1] for i in range(len(test_generator))])

print("balanced accuracy", test_dict['balanced_acc_m'])
print("aupr", average_precision_score(test_labels_, test_preds, average="micro"))
print("recall", test_dict['recall_m'], "precision", test_dict['precision_m'], "\n")

106
149
200
71/71 [==============================] - 59s 831ms/step - loss: 0.7326 - balanced_acc_m: 0.5915 - pos_pred_m: 56.4648 - neg_pred_m: 199.5352 - tp_m: 42.6197 - fp_m: 13.8451 - tn_m: 101.3521 - fn_m: 98.1831 - recall_m: 0.3031 - neg_recall_m: 0.8799 - precision_m: 0.7546 - neg_precision_m: 0.5081
balanced accuracy 0.5915223360061646
aupr 0.5466265317308371
recall 0.3031086027622223 precision 0.7546137571334839 

